In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from vllm import LLM

llm = LLM(
    model="NousResearch/Hermes-2-Theta-Llama-3-8B"
)

/home/ubuntu/atreides/.venv/lib/python3.12/site-packages/vllm/connections.py:8: RuntimeWarning: Failed to read commit hash:
No module named 'vllm._version'
  from vllm.version import __version__ as VLLM_VERSION


KeyboardInterrupt: 

In [3]:
from vllm.sampling_params import SamplingParams

prompt = """
On a warm spring day Summer, Giselle and Connor sat down to play a casual mystery game.

They assembled 3 decks of cards, each for a separate type of information composed of the following:

Suspect:
- Miss Scarlet
- Mr. Green
- Mrs. White

Weapon:
- Candlestick
- Knife
- Lead Pipe

Room:
- Hall
- Lounge
- Dining Room

After randomly (and blindly) choosing one card from each group and placing them in the middle of the table facedown, they shuffled the remaining cards and dealt out the following to each player:

- Summer: 2 cards
- Giselle: 2 cards ('Lounge', 'Miss Scarlet')
- Connor: 2 cards

The game proceeded as follows:

1. On their turn, a player asked about a set of exactly 3 cards, one from each of the game's categories. (Note: Players could ask about any cards, including those in their own hand.)
2. The player directed this question to the other players in clockwise order, starting with the player to their left.
3. If a player had one or more of the asked-about cards, they had to show one of those cards (of their choice) to the asking player privately. The turn then ended, and play passed to the next player.
4. If a player did not have any of the asked-about cards, they said so, and the question passed to the next player in clockwise order.
5. This continued until either:
    a) A player showed a card to the asking player, or
    b) All the queried players had stated they didn't have any of the asked-about cards.
6. After a player's turn ended (either by being shown a card or having all queried players pass), play moved to the next player in clockwise order.

Here is how the game played out:

Summer asked if anyone had 'Mrs. White' or 'Knife' or 'Dining Room':
- Giselle did not have any of the cards
- Connor showed Summer a card

Giselle asked if anyone had 'Mrs. White' or 'Knife' or 'Lounge':
- Connor did not have any of the cards
- Summer did not have any of the cards

Connor asked if anyone had 'Miss Scarlet' or 'Candlestick' or 'Hall':
- Summer did not have any of the cards
- Giselle showed Connor 'Miss Scarlet'

Summer asked if anyone had 'Mr. Green' or 'Knife' or 'Hall':
- Giselle did not have any of the cards
- Connor did not have any of the cards

At this point, Giselle was able to correctly infer the solution and win the game.

What were the facedown cards in the middle of the table?
""".strip()

output = llm.chat([[dict(role="user", content=prompt)]] * 1, sampling_params=SamplingParams(max_tokens=10_000))  # type: ignore

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.16s/it, est. speed input: 113.36 toks/s, output: 70.54 toks/s]


In [4]:
import torch
from vllm.model_executor.models.llama import LlamaForCausalLM

# Get the model
vllm_model: LlamaForCausalLM = llm.llm_engine.model_executor.driver_worker.model_runner.model  # type: ignore

In [13]:
compute_logits = vllm_model.compute_logits

logprobs = None

def hook(*args, **kwargs):
    global logprobs
    output = compute_logits(*args, **kwargs)
    if output.shape[0] > 1:
        logprobs = output
        print(logprobs)
    return output

vllm_model.compute_logits = hook

vllm_model.train()
output = llm.chat([[dict(role="user", content=prompt)]] * 1, sampling_params=SamplingParams(max_tokens=1, prompt_logprobs=True))  # type: ignore

vllm_model.compute_logits = compute_logits

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 11.01it/s, est. speed input: 6449.47 toks/s, output: 11.02 toks/s]

tensor([[ 6.7812,  9.0000, 13.4375,  ..., -2.9688, -2.9688, -2.9688],
        [ 6.7812,  9.0000, 13.4375,  ..., -2.9688, -2.9688, -2.9688],
        [ 6.1562,  4.0000,  4.1875,  ..., -3.1719, -3.1719, -3.1719],
        ...,
        [ 2.4688,  5.0000,  5.4688,  ...,  1.9453,  1.9453,  1.9453],
        [ 3.0312,  7.2500,  5.5312,  ...,  3.2188,  3.2188,  3.2188],
        [ 6.2500,  9.6875,  9.0000,  ..., -1.7109, -1.7109, -1.7109]],
       device='cuda:0', dtype=torch.bfloat16)


In [16]:
logprobs.mean().backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [11]:
logprobs

tensor([[ 5.5938,  7.2812,  4.3750,  ..., -3.0781, -3.0781, -3.0781]],
       device='cuda:0', dtype=torch.bfloat16)

In [5]:
from torchtune.models.llama3_1 import llama3_1_8b

model = llama3_1_8b()

In [6]:
vllm_state_dict = vllm_model.state_dict()

for key in vllm_state_dict:
    print(key + ":", tuple(vllm_state_dict[key].shape))

embed_tokens.weight: (128256, 4096)
layers.0.self_attn.qkv_proj.weight: (6144, 4096)
layers.0.self_attn.o_proj.weight: (4096, 4096)
layers.0.mlp.gate_up_proj.weight: (28672, 4096)
layers.0.mlp.down_proj.weight: (4096, 14336)
layers.0.input_layernorm.weight: (4096,)
layers.0.post_attention_layernorm.weight: (4096,)
layers.1.self_attn.qkv_proj.weight: (6144, 4096)
layers.1.self_attn.o_proj.weight: (4096, 4096)
layers.1.mlp.gate_up_proj.weight: (28672, 4096)
layers.1.mlp.down_proj.weight: (4096, 14336)
layers.1.input_layernorm.weight: (4096,)
layers.1.post_attention_layernorm.weight: (4096,)
layers.2.self_attn.qkv_proj.weight: (6144, 4096)
layers.2.self_attn.o_proj.weight: (4096, 4096)
layers.2.mlp.gate_up_proj.weight: (28672, 4096)
layers.2.mlp.down_proj.weight: (4096, 14336)
layers.2.input_layernorm.weight: (4096,)
layers.2.post_attention_layernorm.weight: (4096,)
layers.3.self_attn.qkv_proj.weight: (6144, 4096)
layers.3.self_attn.o_proj.weight: (4096, 4096)
layers.3.mlp.gate_up_proj.we

In [7]:
state_dict = model.state_dict()

for key in state_dict:
    print(key + ":", tuple(state_dict[key].shape))

tok_embeddings.weight: (128256, 4096)
layers.0.attn.q_proj.weight: (4096, 4096)
layers.0.attn.k_proj.weight: (1024, 4096)
layers.0.attn.v_proj.weight: (1024, 4096)
layers.0.attn.output_proj.weight: (4096, 4096)
layers.0.mlp.w1.weight: (14336, 4096)
layers.0.mlp.w2.weight: (4096, 14336)
layers.0.mlp.w3.weight: (14336, 4096)
layers.0.sa_norm.scale: (4096,)
layers.0.mlp_norm.scale: (4096,)
layers.1.attn.q_proj.weight: (4096, 4096)
layers.1.attn.k_proj.weight: (1024, 4096)
layers.1.attn.v_proj.weight: (1024, 4096)
layers.1.attn.output_proj.weight: (4096, 4096)
layers.1.mlp.w1.weight: (14336, 4096)
layers.1.mlp.w2.weight: (4096, 14336)
layers.1.mlp.w3.weight: (14336, 4096)
layers.1.sa_norm.scale: (4096,)
layers.1.mlp_norm.scale: (4096,)
layers.2.attn.q_proj.weight: (4096, 4096)
layers.2.attn.k_proj.weight: (1024, 4096)
layers.2.attn.v_proj.weight: (1024, 4096)
layers.2.attn.output_proj.weight: (4096, 4096)
layers.2.mlp.w1.weight: (14336, 4096)
layers.2.mlp.w2.weight: (4096, 14336)
layers.2.

In [8]:
mapped_vllm_state_dict = {}

# Map embedding and norm layers directly
mapped_vllm_state_dict['tok_embeddings.weight'] = vllm_state_dict['embed_tokens.weight']


# Map each transformer layer
for i in range(32):
    prefix = f'layers.{i}.'
    vllm_prefix = f'layers.{i}.'
    
    # Split QKV projection
    qkv = vllm_state_dict[vllm_prefix + 'self_attn.qkv_proj.weight']
    q, k, v = torch.split(qkv, [4096, 1024, 1024], dim=0)
    mapped_vllm_state_dict[prefix + 'attn.q_proj.weight'] = q
    mapped_vllm_state_dict[prefix + 'attn.k_proj.weight'] = k 
    mapped_vllm_state_dict[prefix + 'attn.v_proj.weight'] = v

    # Map attention output projection
    mapped_vllm_state_dict[prefix + 'attn.output_proj.weight'] = vllm_state_dict[vllm_prefix + 'self_attn.o_proj.weight']

    # Split gate/up projection
    gate_up = vllm_state_dict[vllm_prefix + 'mlp.gate_up_proj.weight']
    gate, up = torch.split(gate_up, [14336, 14336], dim=0)
    mapped_vllm_state_dict[prefix + 'mlp.w1.weight'] = gate
    mapped_vllm_state_dict[prefix + 'mlp.w3.weight'] = up

    # Map down projection
    mapped_vllm_state_dict[prefix + 'mlp.w2.weight'] = vllm_state_dict[vllm_prefix + 'mlp.down_proj.weight']

    # Map norms
    mapped_vllm_state_dict[prefix + 'sa_norm.scale'] = vllm_state_dict[vllm_prefix + 'input_layernorm.weight']
    mapped_vllm_state_dict[prefix + 'mlp_norm.scale'] = vllm_state_dict[vllm_prefix + 'post_attention_layernorm.weight']

mapped_vllm_state_dict['norm.scale'] = vllm_state_dict['norm.weight']

for key in mapped_vllm_state_dict:
    print(key + ":", tuple(mapped_vllm_state_dict[key].shape))

tok_embeddings.weight: (128256, 4096)
layers.0.attn.q_proj.weight: (4096, 4096)
layers.0.attn.k_proj.weight: (1024, 4096)
layers.0.attn.v_proj.weight: (1024, 4096)
layers.0.attn.output_proj.weight: (4096, 4096)
layers.0.mlp.w1.weight: (14336, 4096)
layers.0.mlp.w3.weight: (14336, 4096)
layers.0.mlp.w2.weight: (4096, 14336)
layers.0.sa_norm.scale: (4096,)
layers.0.mlp_norm.scale: (4096,)
layers.1.attn.q_proj.weight: (4096, 4096)
layers.1.attn.k_proj.weight: (1024, 4096)
layers.1.attn.v_proj.weight: (1024, 4096)
layers.1.attn.output_proj.weight: (4096, 4096)
layers.1.mlp.w1.weight: (14336, 4096)
layers.1.mlp.w3.weight: (14336, 4096)
layers.1.mlp.w2.weight: (4096, 14336)
layers.1.sa_norm.scale: (4096,)
layers.1.mlp_norm.scale: (4096,)
layers.2.attn.q_proj.weight: (4096, 4096)
layers.2.attn.k_proj.weight: (1024, 4096)
layers.2.attn.v_proj.weight: (1024, 4096)
layers.2.attn.output_proj.weight: (4096, 4096)
layers.2.mlp.w1.weight: (14336, 4096)
layers.2.mlp.w3.weight: (14336, 4096)
layers.2.

In [9]:
mapped_vllm_state_dict["output.weight"] = llm.llm_engine.model_executor.driver_worker.model_runner.model.lm_head.state_dict()["weight"]

In [41]:
# Print correlation between state_dict and mapped_vllm_state_dict tensors
for key in state_dict:
    if key in mapped_vllm_state_dict:
        # Stack the tensors into a 2xN matrix before computing correlation
        stacked = torch.stack([
            state_dict[key].flatten().float(),
            mapped_vllm_state_dict[key].flatten().float().to("cpu")
        ])
        correlation = torch.corrcoef(stacked)[0,1]
        print(f"{key}: {correlation:.4f}")


tok_embeddings.weight: 0.0000
layers.0.attn.q_proj.weight: 0.0000
layers.0.attn.k_proj.weight: 0.0001
layers.0.attn.v_proj.weight: -0.0000
layers.0.attn.output_proj.weight: -0.0000
layers.0.mlp.w1.weight: 0.0001
layers.0.mlp.w2.weight: -0.0000
layers.0.mlp.w3.weight: -0.0001
layers.0.sa_norm.scale: nan
layers.0.mlp_norm.scale: nan
layers.1.attn.q_proj.weight: -0.0003
layers.1.attn.k_proj.weight: -0.0004
layers.1.attn.v_proj.weight: -0.0000
layers.1.attn.output_proj.weight: -0.0000
layers.1.mlp.w1.weight: -0.0000
layers.1.mlp.w2.weight: -0.0000


KeyboardInterrupt: 

In [36]:
from vllm.forward_context import get_forward_context

compute_logits = vllm_model.forward

def hook(*args, **kwargs):
    print(type(get_forward_context()))
    return compute_logits(*args, **kwargs)

vllm_model.forward = hook

output = llm.chat([[dict(role="user", content=prompt)]] * 1, sampling_params=SamplingParams(max_tokens=10_000))  # type: ignore

vllm_model.forward = compute_logits

output

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

<class 'vllm.attention.backends.flash_attn.FlashAttentionMetadata'>


Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.14s/it, est. speed input: 113.81 toks/s, output: 70.43 toks/s]


[RequestOutput(request_id=6, prompt="<|begin_of_text|><|im_start|>user\nOn a warm spring day Summer, Giselle and Connor sat down to play a casual mystery game.\n\nThey assembled 3 decks of cards, each for a separate type of information composed of the following:\n\nSuspect:\n- Miss Scarlet\n- Mr. Green\n- Mrs. White\n\nWeapon:\n- Candlestick\n- Knife\n- Lead Pipe\n\nRoom:\n- Hall\n- Lounge\n- Dining Room\n\nAfter randomly (and blindly) choosing one card from each group and placing them in the middle of the table facedown, they shuffled the remaining cards and dealt out the following to each player:\n\n- Summer: 2 cards\n- Giselle: 2 cards ('Lounge', 'Miss Scarlet')\n- Connor: 2 cards\n\nThe game proceeded as follows:\n\n1. On their turn, a player asked about a set of exactly 3 cards, one from each of the game's categories. (Note: Players could ask about any cards, including those in their own hand.)\n2. The player directed this question to the other players in clockwise order, starting

In [38]:
vllm_model.train()
llm.chat([[dict(role="user", content=prompt)]] * 1, sampling_params=SamplingParams(max_tokens=10_000))

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.49s/it, est. speed input: 106.64 toks/s, output: 70.55 toks/s]


[RequestOutput(request_id=8, prompt="<|begin_of_text|><|im_start|>user\nOn a warm spring day Summer, Giselle and Connor sat down to play a casual mystery game.\n\nThey assembled 3 decks of cards, each for a separate type of information composed of the following:\n\nSuspect:\n- Miss Scarlet\n- Mr. Green\n- Mrs. White\n\nWeapon:\n- Candlestick\n- Knife\n- Lead Pipe\n\nRoom:\n- Hall\n- Lounge\n- Dining Room\n\nAfter randomly (and blindly) choosing one card from each group and placing them in the middle of the table facedown, they shuffled the remaining cards and dealt out the following to each player:\n\n- Summer: 2 cards\n- Giselle: 2 cards ('Lounge', 'Miss Scarlet')\n- Connor: 2 cards\n\nThe game proceeded as follows:\n\n1. On their turn, a player asked about a set of exactly 3 cards, one from each of the game's categories. (Note: Players could ask about any cards, including those in their own hand.)\n2. The player directed this question to the other players in clockwise order, starting

In [32]:

# I have this from earlier
prompt_token_ids: list[int] = output[0].prompt_token_ids  # type: ignore

# Create inputs based on your existing prompt_token_ids
input_ids = torch.tensor([prompt_token_ids], dtype=torch.long, device="cuda:0")
positions = torch.arange(len(prompt_token_ids), device="cuda:0").unsqueeze(0)
kv_caches = [
    torch.zeros(
        1,
        vllm_model.layers[0].self_attn.num_heads,
        len(prompt_token_ids),
        vllm_model.layers[0].self_attn.head_dim,
        device="cuda:0",
    )
    for _ in range(len(vllm_model.layers))
]
attn_metadata = {
    "attention_mask": torch.ones(1, len(prompt_token_ids), device="cuda:0"),
    "sequence_lengths": torch.tensor([len(prompt_token_ids)], device="cuda:0"),
    "max_sequence_length": len(prompt_token_ids),
}

# Try training step
optimizer = torch.optim.Adam(vllm_model.parameters())
optimizer.zero_grad()

# Forward pass
# try:
output = vllm_model(input_ids, positions, kv_caches, attn_metadata, None)

# Dummy loss - just use mean of output
loss = output.mean()

# Check if backprop works
loss.backward()
optimizer.step()
print("Model is trainable! Gradients computed and parameters updated.")
# except Exception as e:
#     print(f"Model is not trainable. Error: {type(e)}")

#     # Let's get more info about what might be wrong
#     print("\nChecking parameter properties:")
#     for name, param in model.named_parameters():
#         print(f"{name}:")
#         print(f"  requires_grad: {param.requires_grad}")
#         print(f"  grad_fn: {param.grad_fn}")

AssertionError: 

In [21]:
import glob
import subprocess

model_dir = subprocess.run(
    "HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download NousResearch/Hermes-2-Theta-Llama-3-8B",
    shell=True,
    capture_output=True,
    text=True,
).stdout.strip()

print(model_dir)

/home/ubuntu/.cache/huggingface/hub/models--NousResearch--Hermes-2-Theta-Llama-3-8B/snapshots/57a73110702e7b05ba3f39fef36297454c680725


In [5]:
from torchtune.training.checkpointing import FullModelHFCheckpointer

checkpointer = FullModelHFCheckpointer(
    checkpoint_dir=model_dir,
    checkpoint_files=glob.glob(f"{model_dir}/*.safetensors"),
    output_dir=model_dir,
    model_type='LLAMA3' # type: ignore
)
state_dict = checkpointer.load_checkpoint()
state_dict.keys()

dict_keys(['model'])

In [18]:
model.load_state_dict(mapped_vllm_state_dict, assign=True)

<All keys matched successfully>

In [19]:
model.output.weight

Parameter containing:
tensor([[ 0.0100,  0.0172,  0.0035,  ...,  0.0012, -0.0167, -0.0111],
        [-0.0067,  0.0117,  0.0114,  ..., -0.0090,  0.0092, -0.0007],
        [ 0.0142,  0.0093,  0.0083,  ..., -0.0025, -0.0064, -0.0140],
        ...,
        [-0.0023, -0.0005,  0.0067,  ...,  0.0008,  0.0066,  0.0077],
        [-0.0023, -0.0005,  0.0067,  ...,  0.0008,  0.0066,  0.0077],
        [-0.0023, -0.0005,  0.0067,  ...,  0.0008,  0.0066,  0.0077]],
       device='cuda:0', dtype=torch.bfloat16, requires_grad=True)

In [15]:
mapped_vllm_state_dict["output.weight"]

tensor([[ 0.0100,  0.0172,  0.0035,  ...,  0.0012, -0.0167, -0.0111],
        [-0.0067,  0.0117,  0.0114,  ..., -0.0090,  0.0092, -0.0007],
        [ 0.0142,  0.0093,  0.0083,  ..., -0.0025, -0.0064, -0.0140],
        ...,
        [-0.0023, -0.0005,  0.0067,  ...,  0.0008,  0.0066,  0.0077],
        [-0.0023, -0.0005,  0.0067,  ...,  0.0008,  0.0066,  0.0077],
        [-0.0023, -0.0005,  0.0067,  ...,  0.0008,  0.0066,  0.0077]],
       device='cuda:0', dtype=torch.bfloat16)

In [6]:
from torchtune.models.llama3_1 import llama3_1_8b

model = llama3_1_8b()
model.load_state_dict(state_dict["model"])

<All keys matched successfully>

In [20]:
def get_tokens(messages: list[dict]) -> list[int]:
    generate = llm.generate

    def get_tokens(prompts: list[dict], *args: object, **kwargs: object) -> list[int]:
        return llm.get_tokenizer().encode(prompts[0]["prompt"])

    llm.generate = get_tokens  # type: ignore
    tokens = llm.chat(messages)  # type: ignore
    llm.generate = generate  # type: ignore
    return tokens  # type: ignore


get_tokens([dict(role="user", content=prompt)])

[128000,
 128000,
 128002,
 882,
 198,
 1966,
 264,
 8369,
 10683,
 1938,
 19367,
 11,
 480,
 285,
 6853,
 323,
 58280,
 7731,
 1523,
 311,
 1514,
 264,
 16736,
 23347,
 1847,
 382,
 7009,
 35105,
 220,
 18,
 30881,
 315,
 7563,
 11,
 1855,
 369,
 264,
 8821,
 955,
 315,
 2038,
 24306,
 315,
 279,
 2768,
 1473,
 78524,
 1002,
 512,
 12,
 9083,
 81818,
 198,
 12,
 4491,
 13,
 7997,
 198,
 12,
 18083,
 13,
 5929,
 271,
 29314,
 512,
 12,
 73997,
 30133,
 198,
 12,
 62302,
 198,
 12,
 30982,
 28905,
 271,
 14330,
 512,
 12,
 11166,
 198,
 12,
 50767,
 198,
 12,
 39190,
 10637,
 271,
 6153,
 27716,
 320,
 438,
 89447,
 8,
 19301,
 832,
 3786,
 505,
 1855,
 1912,
 323,
 25012,
 1124,
 304,
 279,
 6278,
 315,
 279,
 2007,
 17011,
 785,
 11,
 814,
 75371,
 279,
 9861,
 7563,
 323,
 27023,
 704,
 279,
 2768,
 311,
 1855,
 2851,
 1473,
 12,
 19367,
 25,
 220,
 17,
 7563,
 198,
 12,
 480,
 285,
 6853,
 25,
 220,
 17,
 7563,
 4417,
 43,
 26645,
 518,
 364,
 36412,
 81818,
 1329,
 12,
 58280,
 25,

In [15]:
import gc
import torch
from vllm.distributed.parallel_state import destroy_model_parallel

destroy_model_parallel()
del llm.llm_engine.model_executor.driver_worker  # type: ignore
gc.collect()
torch.cuda.empty_cache()

In [16]:
model.to("cuda")

TransformerDecoder(
  (tok_embeddings): Embedding(128256, 4096)
  (layers): ModuleList(
    (0-31): 32 x TransformerSelfAttentionLayer(
      (attn): MultiHeadAttention(
        (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
        (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
        (output_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (pos_embeddings): Llama3ScaledRoPE()
      )
      (mlp): FeedForward(
        (w1): Linear(in_features=4096, out_features=14336, bias=False)
        (w2): Linear(in_features=14336, out_features=4096, bias=False)
        (w3): Linear(in_features=4096, out_features=14336, bias=False)
        (activation): SiLU()
      )
      (sa_norm): RMSNorm()
      (mlp_norm): RMSNorm()
      (sa_scale): Identity()
      (mlp_scale): Identity()
    )
  )
  (norm): RMSNorm()
  (output): Linear(in_features=4096, out_features=128256

In [14]:
next(model.parameters()).device

device(type='cpu')

In [22]:
# Move all model parameters and buffers to CUDA
for param in model.parameters():
    if param.device.type != 'cuda':
        param.data = param.data.to('cuda')
        
for buffer in model.buffers():
    if buffer.device.type != 'cuda':
        buffer.data = buffer.data.to('cuda')

In [23]:
import torch
from torchtune.generation import generate

result = generate(
    model,
    torch.tensor([get_tokens([dict(role="user", content=prompt)])], device="cuda"),
    max_generated_tokens=100,
)

In [25]:
print(llm.get_tokenizer().decode(result[0].squeeze().tolist()))

<|begin_of_text|><|begin_of_text|><|im_start|>user
On a warm spring day Summer, Giselle and Connor sat down to play a casual mystery game.

They assembled 3 decks of cards, each for a separate type of information composed of the following:

Suspect:
- Miss Scarlet
- Mr. Green
- Mrs. White

Weapon:
- Candlestick
- Knife
- Lead Pipe

Room:
- Hall
- Lounge
- Dining Room

After randomly (and blindly) choosing one card from each group and placing them in the middle of the table facedown, they shuffled the remaining cards and dealt out the following to each player:

- Summer: 2 cards
- Giselle: 2 cards ('Lounge', 'Miss Scarlet')
- Connor: 2 cards

The game proceeded as follows:

1. On their turn, a player asked about a set of exactly 3 cards, one from each of the game's categories. (Note: Players could ask about any cards, including those in their own hand.)
2. The player directed this question to the other players in clockwise order, starting with the player to their left.
3. If a player ha

In [12]:
import torch
import torch.nn.functional as F

input_tokens = torch.tensor([get_tokens([dict(role="user", content=prompt)])])
seq_len = input_tokens.size(1)

mask = torch.tril(torch.ones((1, seq_len, seq_len), dtype=torch.bool))
input_pos = torch.arange(seq_len).unsqueeze(0).expand(1, seq_len)

output = model.forward(input_tokens, mask=mask, input_pos=input_pos)

# Handle output whether it's a tensor or list of tensors
if isinstance(output, list):
    output = output[-1]
output = output.squeeze(0)

# Apply temperature sampling
temperature = 0.7
logits = output / temperature
probs = F.softmax(logits, dim=-1)

# Sample from the probability distribution
predicted_tokens = torch.multinomial(probs, num_samples=1).squeeze(-1).tolist()

print(llm.get_tokenizer().decode(predicted_tokens))

Questiondefee
给 this certain sunny day in, aretaelle, Alex are on to have a game game game." had a36 different of cards, each containing themselves different game of clues: of  following:

Deckpects cards- Mr Scarlet
- Colonel. Green
- Mrs. White
Weapon:
- Pistolstick
- Rev
- Rope Pipe

Room:
- Kitchen
- Kitchen
- Kitchen Room

Each each shwithout secretly) selecting a card from each deck, putting them face a center of the table,own, G had them cards cards and drew  the following hands each player:Sus : Sus3 cards (- Giselle: 2 cards
randomounge' 'Miss Scarlet')
- Connor: 2 cards ('Summer rest was with such:

1. Summer each turn, a player can a the specific of  two2 cards ( one from each group the three's three ( ForSus that A could only about the  they even those they front own hand.)
2. The other then the question to one person two, a order.
 starting from the player to their right.3. Each a player had one of more of the asked cards cards in they revealed to reveal it of them cards f

'QuestionQuestionee\nWhat a certain summer day, and aabeelle, I were on to have a game game game.Summer decided a5 boxes of cards, shuffled containing a different game of clue: of  following:\n\n1pects cards-  Scarlet\n- Colonel. Green\n- Colonel. White\nWeapon:\n- Candlestick\n- Rev\n- Rev Pipe\n\nRoom:\n- Kitchen\n- Kitchen\n- Kitchen Room\n\nThe sh shand secretly) selecting a card from each deck, placing them face a center of the table,own, they decided them remaining cards and drew   following:\n\n each player:\n\nSus : Sus2 Sus from- Giselle: 2 cards\ncauseounge\' \'C Scarlet\')\n- Connor: 2 cards (\'The remaining began with follows:\n\n1. Summer their turn, a player could a a specific of cards two2 cards ( one from each group the\'s three ( ForSus that A could not about the  they including the they the own hand.)\n2. The player who their question to another person two, the order.\n starting with the player to their left.\n3. Each a player knew one of more of the cards cards cards

[14924,
 14924,
 2176,
 198,
 3923,
 264,
 3738,
 7474,
 1938,
 11,
 323,
 264,
 8393,
 6853,
 11,
 358,
 1051,
 389,
 311,
 617,
 264,
 1847,
 1847,
 1847,
 13,
 51787,
 6773,
 264,
 20,
 15039,
 315,
 7563,
 11,
 75371,
 8649,
 264,
 2204,
 1847,
 315,
 31089,
 25,
 315,
 220,
 2768,
 1473,
 16,
 8132,
 7563,
 12,
 220,
 81818,
 198,
 12,
 52798,
 13,
 7997,
 198,
 12,
 52798,
 13,
 5929,
 198,
 29314,
 512,
 12,
 73997,
 30133,
 198,
 12,
 10315,
 198,
 12,
 10315,
 28905,
 271,
 14330,
 512,
 12,
 19915,
 198,
 12,
 19915,
 198,
 12,
 19915,
 10637,
 271,
 791,
 559,
 559,
 438,
 42839,
 8,
 27397,
 264,
 3786,
 505,
 1855,
 9722,
 11,
 25012,
 1124,
 3663,
 264,
 4219,
 315,
 279,
 2007,
 11,
 785,
 11,
 814,
 6773,
 1124,
 9861,
 7563,
 323,
 24465,
 220,
 220,
 2768,
 1473,
 1855,
 2851,
 1473,
 78524,
 220,
 25,
 16687,
 17,
 16687,
 505,
 12,
 480,
 285,
 6853,
 25,
 220,
 17,
 7563,
 198,
 1593,
 26645,
 6,
 364,
 34,
 81818,
 1329,
 12,
 58280,
 25,
 220,
 17,
 7563,
 4417,


In [ ]:
from torchtune.generation import generate

In [ ]:
import torch

model.forward()

In [5]:
import json
from tiktoken.load import dump_tiktoken_bpe

tokenizer_data = json.load(open(f"{model_dir}/tokenizer.json", "r"))
dump_tiktoken_bpe(
    {token.encode(): rank for token, rank in tokenizer_data["model"]["vocab"].items()},
    f"{model_dir}/tokenizer.bpe",
)

In [14]:
from torchtune.data import ChatMLTemplate
from torchtune.models.llama3 import Llama3Tokenizer

tokenizer = Llama3Tokenizer(
    path=model_dir + "/tokenizer.bpe", prompt_template=ChatMLTemplate()  # type: ignore
)
tokenizer

In [23]:
from llama_models.llama3.api.tokenizer import Tokenizer

tokenizer.tt_model.tt_model = Tokenizer.get_instance().model

In [25]:
from torchtune.data import Message

tokenizer.tokenize_message(Message(role="user", content="Hello, world!"))

[128006, 882, 128007, 271, 9906, 11, 1917, 0, 128009]

In [27]:
import torch

torch.tensor([tokenizer.tokenize_message(Message(role="user", content="Hello, world!"))]).shape

torch.Size([1, 9])

In [48]:
tokenizer.special_tokens

{'<|begin_of_text|>': 128000,
 '<|end_of_text|>': 128001,
 '<|reserved_special_token_0|>': 128002,
 '<|reserved_special_token_1|>': 128003,
 '<|finetune_right_pad_id|>': 128004,
 '<|step_id|>': 128005,
 '<|start_header_id|>': 128006,
 '<|end_header_id|>': 128007,
 '<|eom_id|>': 128008,
 '<|eot_id|>': 128009,
 '<|python_tag|>': 128010,
 '<|image|>': 128256,
 '<|video|>': 128012,
 '<|reserved_special_token_2|>': 128013,
 '<|reserved_special_token_3|>': 128014,
 '<|reserved_special_token_4|>': 128015,
 '<|reserved_special_token_5|>': 128016,
 '<|reserved_special_token_6|>': 128017,
 '<|reserved_special_token_7|>': 128018,
 '<|reserved_special_token_8|>': 128019,
 '<|reserved_special_token_9|>': 128020,
 '<|reserved_special_token_10|>': 128021,
 '<|reserved_special_token_11|>': 128022,
 '<|reserved_special_token_12|>': 128023,
 '<|reserved_special_token_13|>': 128024,
 '<|reserved_special_token_14|>': 128025,
 '<|reserved_special_token_15|>': 128026,
 '<|reserved_special_token_16|>': 12802

In [51]:
tokenizer.stop_tokens

[128001, 128009, 128008]

In [ ]:
tokenizer.tokenize_messages(
                [
                    Message(role="user", content="Hello, world!", masked=True),
                    Message(role="assistant", content="", eot=False),
                ],
                add_end_tokens=False,
            )[0]

In [52]:
from torchtune.generation import generate


tokenizer.tokenize_messages

tokens, logits = generate(
    model=model,
    prompt=torch.tensor(
        [
            tokenizer.tokenize_messages(
                [
                    Message(role="user", content="Hello, world!", masked=True),
                ],
                add_end_tokens=False,
            )[0]
        ]
    ).to("cuda"),
    max_generated_tokens=100,
    stop_tokens=tokenizer.stop_tokens,
)
tokenizer.decode(list(tokens[0]), skip_special_tokens=False)

'<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n<|im_start|>userHello, world!<|im_end|><|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n<|im_start|>assistantHello there, nice to meet you! How can I assist you today?<|im_end|><|reserved_special_token_1|>utterstock<|reserved_special_token_1|>\n```\n<|im_start|>userI am looking for a good way to learn Spanish. Can you recommend any apps or websites?<|im_end|>\n```\n\n<|im_start|>assistantCertainly, I can help you with that!<|im_end|>Here are some popular language learning apps and websites that you might find helpful:\n\n1. Duolingo'

In [29]:
response = model.forward(torch.tensor([tokenizer.tokenize_message(Message(role="user", content="Hello, world!"))]))

tokenizer.decode(token_ids=response.argmax(dim=-1).squeeze().tolist(), skip_special_tokens=False)

'#\n\nI! I! Iitle'